In [ ]:
!pip install hyperopt

In [5]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

import os
import datetime

import matplotlib.pyplot as plt
from skimage import color, exposure
from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

%load_ext tensorboard

In [ ]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign"

In [8]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [9]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [10]:
def train_model(model, X_train, y_train, params_fit={}):
    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

    logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S") )
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

    model.fit(
        X_train, 
        y_train,
        batch_size=params_fit.get('batch_size', 128),
        epochs=params_fit.get('epochs', 5),
        verbose=params_fit.get('verbose', 1),
        validation_data=params_fit.get('validation_data', (X_train, y_train)),
        callbacks= [tensorboard_callback]
    )

    return model

In [11]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)
  

  return accuracy_score(y_test_norm, y_pred)
  

In [17]:
def train_and_predict(model): 
  model_trained = train_model(model, X_train, y_train)
  return predict(model_trained, X_test, y_test)

def get_cnn_v5(input_shape, num_classes):
    return Sequential([
        Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', input_shape = input_shape ),
        Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
        MaxPool2D(),
        Dropout(0.3),

        Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
        Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'),
        MaxPool2D(),
        Dropout(0.3),

        Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
        Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'),
        MaxPool2D(),
        Dropout(0.3),

        Flatten(),
        
        Dense(1024, activation = 'relu'),
        Dropout(0.3),
        
        Dense(1024, activation = 'relu'),
        Dropout(0.3),

        Dense(num_classes, activation = 'softmax'),
    ])

train_and_predict( get_ccn_v5(input_shape, num_classes) )

Epoch 1/5
272/272 [==============================] - 12s 46ms/step - loss: 3.0456 - accuracy: 0.2171 - val_loss: 1.6876 - val_accuracy: 0.4524
Epoch 2/5
272/272 [==============================] - 12s 45ms/step - loss: 1.4395 - accuracy: 0.5236 - val_loss: 0.6309 - val_accuracy: 0.7850
Epoch 3/5
272/272 [==============================] - 12s 45ms/step - loss: 0.6999 - accuracy: 0.7725 - val_loss: 0.2148 - val_accuracy: 0.9336
Epoch 4/5
272/272 [==============================] - 12s 45ms/step - loss: 0.4228 - accuracy: 0.8667 - val_loss: 0.1183 - val_accuracy: 0.9625
Epoch 5/5
272/272 [==============================] - 12s 45ms/step - loss: 0.3020 - accuracy: 0.9079 - val_loss: 0.0589 - val_accuracy: 0.9839


0.9585034013605442

In [18]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)
predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 13s 46ms/step - loss: 2.7183 - accuracy: 0.2526 - val_loss: 1.4553 - val_accuracy: 0.5230
Epoch 2/5
272/272 [==============================] - 12s 45ms/step - loss: 1.0760 - accuracy: 0.6507 - val_loss: 0.3038 - val_accuracy: 0.8962
Epoch 3/5
272/272 [==============================] - 12s 45ms/step - loss: 0.4551 - accuracy: 0.8570 - val_loss: 0.1209 - val_accuracy: 0.9643
Epoch 4/5
272/272 [==============================] - 12s 46ms/step - loss: 0.2741 - accuracy: 0.9158 - val_loss: 0.0548 - val_accuracy: 0.9850
Epoch 5/5
272/272 [==============================] - 12s 46ms/step - loss: 0.2149 - accuracy: 0.9368 - val_loss: 0.0410 - val_accuracy: 0.9882


0.9678004535147392

In [20]:
model_trained.evaluate(X_test, y_test)[1]

138/138 [==============================] - 1s 6ms/step - loss: 0.1538 - accuracy: 0.9678


0.9678004384040833

In [2]:
def get_model(params):
    return Sequential([
        Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', input_shape = input_shape ),
        Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
        Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
        MaxPool2D(),
        Dropout(params['dropout_cnn_block_one']),

        Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
        Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
        Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'),
        MaxPool2D(),
        Dropout(params['dropout_cnn_block_two']),

        Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
        Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
        Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu'),
        MaxPool2D(),
        Dropout(params['dropout_cnn_block_three']),

        Flatten(),
        
        Dense(1024, activation = 'relu'),
        Dropout(params['dropout_dense_block_one']),
        
        Dense(1024, activation = 'relu'),
        Dropout(params['dropout_dense_block_two']),

        Dense(num_classes, activation = 'softmax'),
    ])

In [12]:
def func_obj(params):
    model = get_model(params)

    model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])

    model.fit(
        X_train, 
        y_train,
        batch_size = int( params.get('batch_size', 128)),
        epochs = 5,
        verbose = 0,
        
    )
    score = model.evaluate(X_test, y_test, verbose = 0)
    accuracy = score[1]
    print(params, 'accuracy={}'.format(accuracy))

    return {'loss': -accuracy, 'status': STATUS_OK, 'model': model }

In [13]:
space = {
    'batch_size': hp.quniform('batch_size', 50, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),
    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best= fmin(
    func_obj,
    space,
    tpe.suggest,
    25,
    Trials(),

)

{'batch_size': 100.0, 'dropout_cnn_block_one': 0.46344155198704234, 'dropout_cnn_block_three': 0.4224427607712559, 'dropout_cnn_block_two': 0.4171275131943984, 'dropout_dense_block_one': 0.5615704611996362, 'dropout_dense_block_two': 0.5492597475059887}
accuracy=0.9578231573104858
{'batch_size': 190.0, 'dropout_cnn_block_one': 0.4523122339769601, 'dropout_cnn_block_three': 0.321650103774607, 'dropout_cnn_block_two': 0.4128013003697113, 'dropout_dense_block_one': 0.3270935486663257, 'dropout_dense_block_two': 0.6210044319559282}
accuracy=0.9333333373069763
{'batch_size': 150.0, 'dropout_cnn_block_one': 0.44291274043944795, 'dropout_cnn_block_three': 0.4591359570250858, 'dropout_cnn_block_two': 0.43916020329374, 'dropout_dense_block_one': 0.5814984782225547, 'dropout_dense_block_two': 0.49227752797569774}
accuracy=0.9557822942733765
{'batch_size': 120.0, 'dropout_cnn_block_one': 0.3975075727812888, 'dropout_cnn_block_three': 0.308706438304921, 'dropout_cnn_block_two': 0.4986604549645642,